<a href="https://colab.research.google.com/github/kassendra48-pro/Video-Upscale/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4k Video Upscaler Colab (Real-ESRGAN)

Adapted from: [Real-ESRGAN](https://github.com/xinntao/Real-ESRGAN)

Made with ❤️ by: [yuvraj108c](https://github.com/yuvraj108c)

Github repository: https://github.com/yuvraj108c/4k-video-upscaler-colab

# 1. Setup (~1 minute)

In [1]:
# --- 1. SETUP CELL ---
# (Run this cell first, only once per session)

import torch
import os

# Check for GPU
assert torch.cuda.is_available(), "GPU not detected. Please go to Runtime > Change runtime type and select 'GPU'."

# Clone the Real-ESRGAN repository from GitHub
if not os.path.exists('Real-ESRGAN'):
    !git clone https://github.com/xinntao/Real-ESRGAN.git

# Change the current directory to the cloned folder
%cd Real-ESRGAN

# Install the required Python packages
!pip install -q basicsr facexlib gfpgan
!pip install -q -r requirements.txt
!python setup.py develop

# Download the pre-trained models if they don't exist
models_dir = 'experiments/pretrained_models'
os.makedirs(models_dir, exist_ok=True)

models_to_download = {
    "RealESRGAN_x4plus.pth": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth",
    "RealESRGAN_x4plus_anime_6B.pth": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth",
    "realesr-animevideov3.pth": "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/realesr-animevideov3.pth"
}

for model_name, url in models_to_download.items():
    model_path = os.path.join(models_dir, model_name)
    if not os.path.exists(model_path):
        print(f"Downloading {model_name}...")
        !wget {url} -P {models_dir}
    else:
        print(f"{model_name} already exists.")

print("\n✅ Setup complete. You can now run the Mount Drive cell.")

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 759 (delta 106), reused 98 (delta 98), pack-reused 638 (from 1)
Receiving objects: 100% (759/759), 5.38 MiB | 27.82 MiB/s, done.
Resolving deltas: 100% (415/415), done.
/content/Real-ESRGAN
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.6/299.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.0 MB/s et


# 2. Mount drive (optional)

In [2]:
# --- 2. MOUNT DRIVE CELL ---
# (Run this after setup to connect to your Google Drive)

from google.colab import drive
import os

try:
    drive.mount('/content/gdrive')
    print("✅ Google Drive mounted successfully.")
    # Create a symlink for easier access
    if not os.path.exists('/content/MyDrive'):
        os.symlink('/content/gdrive/MyDrive', '/content/MyDrive')
        print("Created a shortcut at /content/MyDrive")
except Exception as e:
    print(f"❌ Error mounting Google Drive: {e}")



Mounted at /content/gdrive
✅ Google Drive mounted successfully.
Created a shortcut at /content/MyDrive


In [3]:
# --- Find Video Bitrate Cell ---
video_path = "/content/gdrive/MyDrive/video.mp4" # <--- Make sure this is your video path
!ffprobe -v error -select_streams v:0 -show_entries stream=bit_rate -of default=noprint_wrappers=1:nokey=1 "{video_path}"

18159718


# 3. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [ ]:
# --- 3. UPSCALE VIDEO CELL ---
# (Run this after mounting drive. Change parameters below as needed.)

import os
import cv2
import subprocess
import sys

# --- FIX: Change to the correct directory every time you run this cell ---
%cd /content/Real-ESRGAN

# --- FIX 1: Reinstall compatible core libraries ---
# This ensures a clean environment for the script to run in.
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121
!pip uninstall -y -q basicsr
# Added ffmpeg-python to the install list to resolve the import error
!pip install -q basicsr==1.4.2 facexlib gfpgan ffmpeg-python

# --- FIX 2: Directly patch the incompatible import statement ---
# This is the most robust way to fix the 'functional_tensor' error.
python_version = f"python{sys.version_info.major}.{sys.version_info.minor}"
file_to_patch = f"/usr/local/lib/{python_version}/dist-packages/basicsr/data/degradations.py"
if os.path.exists(file_to_patch):
    !sed -i 's/from torchvision.transforms.functional_tensor import/from torchvision.transforms.functional import/g' {file_to_patch}
    print("Patched basicsr to fix import error.")

# --- Your video parameters ---
video_path="/content/gdrive/MyDrive/video.mp4" #@param{type:"string"}
output_dir="/content/gdrive/MyDrive/UpscaledVideos" #@param{type:"string"}
resolution = "2 x original" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]
bitrate = "50M" #@param {type:"string"}
convert_to_hdr = True #@param {type:"boolean"}
face_enhance = True #@param {type:"boolean"}

# --- The rest of your code ---
os.makedirs(output_dir, exist_ok=True)
assert os.path.exists(video_path), f"Video file not found at: {video_path}"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
video_capture.release()

final_width, final_height = None, None
aspect_ratio = float(video_width/video_height)

match resolution:
  case "FHD (1920 x 1080)":
    final_width, final_height = 1920, 1080
  case "2k (2560 x 1440)":
    final_width, final_height = 2560, 1440
  case "4k (3840 x 2160)":
    final_width, final_height = 3840, 2160
  case "2 x original":
    final_width, final_height = 2*video_width, 2*video_height
  case "3 x original":
    final_width, final_height = 3*video_width, 3*video_height
  case "4 x original":
    final_width, final_height = 4*video_width, 4*video_height

if aspect_ratio < 1.0 and "original" not in resolution:
  final_width, final_height = final_height, final_width

scale_factor = max(final_width/video_width, final_height/video_height)
while int(video_width * scale_factor) % 2 != 0 or int(video_height * scale_factor) % 2 != 0:
  scale_factor += 0.001

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, using scale_factor={scale_factor:.4f}")

inference_output_dir = "/content/results"
face_enhance_flag = "--face_enhance" if face_enhance else ""

# --- FIX 3: Run the script as a separate process to avoid circular import errors ---
!python inference_realesrgan_video.py \
    -n {model} \
    -i "{video_path}" \
    -o "{inference_output_dir}" \
    --outscale {scale_factor} \
    {face_enhance_flag}

video_name_with_ext = os.path.basename(video_path)
video_name = os.path.splitext(video_name_with_ext)[0]
upscaled_video_path = f"{inference_output_dir}/{video_name}_out.mp4"

if os.path.exists(upscaled_video_path):
    hdr_tag = "_HDR" if convert_to_hdr else ""
    final_video_name = f"{video_name}_upscaled_{final_width}x{final_height}{hdr_tag}.mp4"
    final_video_path = os.path.join(output_dir, final_video_name)

    print("Processing final video with ffmpeg...")
    command = f"ffmpeg -loglevel error -y -i '{upscaled_video_path}'"

    filters = [f"crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2"]

    if convert_to_hdr:
        print("Applying SDR to HDR conversion filters.")
        hdr_filter_chain = (
            "zscale=t=linear:npl=100,"
            "tonemap=tonemap=hable:desat=0,"
            "zscale=p=bt709:t=bt709:m=bt709:r=tv,"
            "zscale=p=bt2020:t=smpte2084:m=bt2020nc"
        )
        filters.append(hdr_filter_chain)
        command += f" -vf \"{','.join(filters)}\""
        command += f" -c:v libx265 -preset medium -crf 22 -pix_fmt yuv420p10le"
        command += f" -x265-params 'keyint=24:bframes=3:b-adapt=2:colorprim=bt2020:transfer=smpte2084:colormatrix=bt2020nc'"
        command += f" -tag:v hvc1"
    else:
        command += f" -vf \"{','.join(filters)}\""
        command += f" -c:v libx264 -preset medium -crf 22 -pix_fmt yuv420p"

    command += f" -b:v {bitrate}"
    command += f" -c:a aac -b:a 320k '{final_video_path}'"

    subprocess.run(command, shell=True)

    print(f"✅ Upscaled video saved to: {final_video_path}")
    if os.path.exists(inference_output_dir):
        !rm -r {inference_output_dir}
else:
    print(f"❌ Error: Upscaled file not found. The Real-ESRGAN inference script may have failed.")


/content/Real-ESRGAN
Patched basicsr to fix import error.
Upscaling from 1920x1080 to 3840x2160, using scale_factor=2.0000
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:01<00:00, 59.4MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth" to /content/Real-ESRGAN/gfpgan/weights/detection_Resnet50_Final.pt

# 4. Disconnect runtime

In [ ]:
from google.colab import runtime

disconnect_when_finish = False  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()